# Naive Bayes for Sentiment Analysis

A tweet or a review can often tell us how its author feels about the subject of the text. In this homework, we will use the [IMDB review dataset](https://ai.stanford.edu/~amaas/data/sentiment/) to automatically infer for a given review whether it's positive, hence its author liked the movie, or negative. This subtype of text classification which aims to infer the sentiments of the text's author is **sentiment analysis**.

Naive Bayes is a probabilistic classifier which assigns among all classes the class with highest probability to a given instance. In our case, we will use it to classify movie review texts into a positive and a negative class. For a given text or document $d$ Naive Bayes decides on the class $\hat{c}$ among all classes $C$ according to the formula:

\begin{align}
\hat{c} &= \underset{c\in C}{\operatorname{argmax}} P(c \mid d)\\
&=\underset{c\in C}{\operatorname{argmax}}\frac{P(d \mid c)P(c)}{P(d)} \\
\end{align}

$P(d)$ is the same for all classes and will not affect which class is more probable, we can disregard it. We will need to calculate the other two quantities likelihood $P(d \mid c)$ and class prior probabilities $P(c)$. 

A movie review $d$ can be represented by features $w_i$ where $i$ indicates word position. Hence, we can rewrite our likelihood for document $d$ with $n$ words as

\begin{align}
P(d \mid c) = P(w_1, w_2, w_3,\dots, w_n \mid c)
\end{align}

We are using the **bag-of-words** assumption. That is, we only care about the words themselves but not their positions. A model with this assumption would evaluate "Apple likes you." and "You like apples." the same after some common text preprocessing steps. Also, we will make the naive Bayes assumption which simplifies the calculation of the likelihood:

\begin{align}
P(d \mid c) &= P(w_1, w_2, w_3,\dots, w_n \mid c)\\ 
&=P(w_1 \mid c)P(w_2 \mid c) \dots P(w_n \mid c)
\end{align}

We will determine the likelihood of the word in a given position $i$ using its count (bag-of-words model). Using the naive Bayes assumption, we will count the occurence of single words (**unigrams**), e.g. "apples". **Bigram**, e.g. "like apples", **trigram** or higher order models violate the naive Bayes assumption. The likelihood will be the fraction of all words in class $c$ that are the same as word $w_i$.

**Note**: For a given word $w_i$ = "apple", $P(w_i = \text{"apple"} \mid c)$ is calculated by counting the occurence of "apple" in **all** documents in class $c$. 

Our final class decision will be made by:

\begin{align}
\hat{c} &= \underset{c\in C}{\operatorname{argmax}}P(c)\underset{0<i\leq n}{\operatorname{\Pi}}P(w_i \mid c)\\
\end{align}

## Sections of The Notebook

1. [Reading Training and Test Sets](#data)<br>
    1.1 [Summary of Dataset](#dataset-summary)<br>
    1.2 [An Example Review](#example-review)<br>
2. [Classification with scikit-learn](#sklearn)<br>
    2.1 [CountVectorizer](#count-vectorizer)<br>
    2.2 [MultinomialNB](#multinomialNB) <br>
    2.3 [Sklearn Pipeline](#sklearn-pipeline)<br>
3. [Processing with SpaCy Language Model](#spacy)<br>
4. [Sklearn with SpaCy Lemmatization and Vocabulary](#sklearn-with-spacy)
5. [Multinomial Naive Bayes Implementation](#multinomialnb-implementation)
6. [Exercises](#exercises)

<a id="data"></a>
## 1. Reading Traning and Test Sets

We will read in the training and testing examples and labels. The examples in this dataset are review texts with some html occasionally. The label information will be deduced from the folder name. Read more about the dataset and folder organization in [README](#dataset-summary).

**Please be aware that the data format and organization will be somewhat different from dataset to dataset. Running the code below as it is will not work on most other datasets.**


In [ ]:
! mkdir data && curl https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz > data/aclImdb_v1.tar.gz

In [ ]:
# tarfile is needed to read from a tar archive
import tarfile

In [ ]:
# path specification for the files
ARCHIVE_NAME = "./data/aclImdb_v1.tar.gz"
README_NAME = "aclImdb/README"
TRAIN_FOLDER = "train/"
TEST_FOLDER = "test/"
POSITIVE_FOLDER = "pos/"
NEGATIVE_FOLDER = "neg/"

In [ ]:
# function definitions for reading data from archive and
# populating training and test sets


def text_decoder(text):
    """Decodes to utf-8"""
    return text.decode()


def truncate_to(text, pos=1000):
    """Show text until the character at pos index"""
    return text[:pos]


def archive_file_contents(tar, info):
    """Get contents of tar file"""
    f = tar.extractfile(info)
    return f.read()


def get_raw_data_from(tar):
    """Reads in the tar archive, forms the training and test set"""

    train_reviews = []
    train_labels = []
    test_reviews = []
    test_labels = []

    # for each file in the archive,
    # get the filename and tarinfo of the compressed file
    for fname, farchive in zip(tar.getnames(), tar.getmembers()):
        # if the file is in the training set
        if TRAIN_FOLDER in fname:
            # and a positive review
            if POSITIVE_FOLDER in fname:
                # add the review to the train_reviews list
                train_reviews.append(text_decoder(archive_file_contents(tar, farchive)))
                # add a 1 to the train_labels list indicating the example is a positive review
                train_labels.append(1)

            # if the file is a negative review
            if NEGATIVE_FOLDER in fname:
                # add the review to the train_reviews list
                train_reviews.append(text_decoder(archive_file_contents(tar, farchive)))
                # add a 0 to the train_labels list indicating the example is a negative review
                train_labels.append(0)

        # if the file is in the test set
        elif TEST_FOLDER in fname:
            # and a positive review
            if POSITIVE_FOLDER in fname:
                # add the review to the test_reviews list
                test_reviews.append(text_decoder(archive_file_contents(tar, farchive)))
                # add a 1 to the test_labels list indicating the example is a positive review
                test_labels.append(1)

            # if the file is a negative review
            if NEGATIVE_FOLDER in fname:
                # add the review to the test_reviews list
                test_reviews.append(text_decoder(archive_file_contents(tar, farchive)))
                # add a 0 to the test_labels list indicating the example is a negative review
                test_labels.append(0)

    return train_reviews, train_labels, test_reviews, test_labels

In [ ]:
with tarfile.open(ARCHIVE_NAME, "r:gz") as tar:
    X_train, y_train, X_test, y_test = get_raw_data_from(tar)

<a id="dataset-summary"></a>
### 1.1 Summary of Dataset

Information about the dataset from the included README file.

In [ ]:
with tarfile.open(ARCHIVE_NAME, "r:gz") as tar:
    readme = text_decoder(archive_file_contents(tar, README_NAME))
    # only the first 1761 characters are informative for us
    print(truncate_to(readme, 1761))

<a id="example-review"></a>
### 1.2 An Example Review

To have an idea of what the reviews look like, we can print the contents of a random review.

In [ ]:
import random

# setting a seed makes sure we get consistent results across runs
random.seed(142)
ind = random.randint(a=0, b=len(X_train))
review_example = X_train[ind]
print(review_example)

<a id="sklearn"></a>
## 2. Classification with sklearn

Scikit-learn ([sklearn](https://scikit-learn.org/stable/)) is a powerful library that simplifies machine learning. Most of the well-known machine learning algorithms are implemented as well as methods for hyperparameter selection, model evaluation, etc. We will use the feature extraction module to calculate word counts, metrics module to calculate accuracies, naive_bayes module for the classification algorithm, and finally pipeline module to put parts of our model together to run more smoothly.

In [ ]:
# necessary imports
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

<a id="count-vectorizer"></a>
### 2.1 CountVectorizer

CountVectorizer returns the number of occurrences of words in the documents. It can take several arguments, which are listed in [its documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). In this section, we will use it with its default arguments. Later, we will specify the vocabulary for which the counts will be calculated, and some other parameters. When the vocabulary is not explicitly specified, it's constructed from the union of all words in all documents.

The shape of the returned counts will be $n_{\text{docs}} \times n_{\text{vocab}}$, where $n_{\text{docs}}$ is the number of documents (in this case just the `review_example`, hence 1), and $n_{\text{vocab}}$ is the size of the vocabulary.

In [ ]:
vectorizer = CountVectorizer()
example_counts = vectorizer.fit_transform(
    [review_example]
)  # CountVectorizer object expects an iterable such as a list

# CountVectorizer returns a sparse representation.
# To see its contents, we will need to cast it
# to a dense array.
example_counts_dense = example_counts.toarray()

# vocabulary is a one-to-one mapping between words and integers
# Each of these integers specify the index of the corresponding word in the count array returned

list(vectorizer.vocabulary_.items())[:10]

In [ ]:
n_vocab = len(list(vectorizer.vocabulary_.items()))
print(f"There are {n_vocab} distinct words in the review_example.")

We can also check which words were the most frequent in `review_example`.

In [ ]:
# Let's find the most frequent word and how many times it occurs in the example review
reverse_vocabulary_lookup = {v: k for k, v in vectorizer.vocabulary_.items()}
len(list(reverse_vocabulary_lookup.items()))

print(
    f"The most frequent word is '{reverse_vocabulary_lookup[np.argmax(example_counts_dense)]}' with {np.max(example_counts_dense)} occurences."
)

Are we gaining any information from the most frequent word? If you change the seed for the random number generator or the index into the training set in [example review section](#example-review), you will see that the most frequent words in the documents are words like "the", "and" which are not informative. These words are called **stop words**, we will clean the documents from stop words in the following sections.

<a id=multinomialNB></a>
### 2.2 MultinomialNB

For our problem, we can use the MultinomialNB (multinomial naive Bayes) implemented in `sklearn`. First, we will need to fit and transform our training data with the `CountVectorizer` instance. Then, we will fit the MultinomialNB classifier to our training dataset, i.e., we will train our MultinomialNB classifier.

In [ ]:
train_counts = vectorizer.fit_transform(X_train)
classifier = MultinomialNB()
classifier.fit(train_counts, y_train)

In [ ]:
test_counts = vectorizer.fit_transform(X_test)

# If you run the two lines of code below you will get a dimension mismatch error.
# See the text before Section 1.3.

# test_predictions = classifier.predict(test_counts)
# accuracy_score(y_test, test_predictions)

In [ ]:
n_train_examples, n_train_corpus = train_counts.shape
n_test_examples, n_test_corpus = test_counts.shape
print(
    f"Training set has {n_train_examples} reviews with a vocabulary of size {n_train_corpus}."
)
print(
    f"Test set has {n_train_examples} reviews with a vocabulary of size {n_test_corpus}."
)

Different vocabularies in the training and test sets lead to "dimension mismatch error". We can fix a vocabulary beforehand and disregard words not in that vocabulary in all of the computations. Instead of specifying the words in the vocabulary explicitly, we will use **SpaCy**'s English model vocabulary.

<a id="sklearn-pipeline"></a>
### 2.3 Sklearn Pipeline

Sklearn allows us to define a custom pipeline where each component of the pipeline can be custom. The inputs to the components are the outputs of the previous component in the pipeline. The code below is equivalent to classification steps in the previous [CountVectorizer](#count-vectorizer) and [MultinomialNB](#multinomialNB) sections.

In [ ]:
pipeline = Pipeline([("vectorize", CountVectorizer()), ("clf", MultinomialNB())])

In [ ]:
pipeline.fit(X_train, y_train)

We will predict the test set classes and report model accuracy on the test set.

In [ ]:
predicted = pipeline.predict(X_test)
acc = accuracy_score(y_test, predicted)
print(f"Test accuracy: {acc}")

<a id="spacy"></a>
## 3. Processing with SpaCy Language Model

Spacy is a Natural Language Processing library in Python. It offers a lot of functionality. We will use its lemmatization method in this notebook for preprocessing the reviews.

We will need to load the English language model. It is in the `shared` folder of the class.

In [ ]:
# first import the SpaCy library
import spacy

nlp = spacy.load("en_core_web_sm-2.3.1", disable=["tagger", "parser", "ner"])

To capture the similarities between words that come from the same root, such as "apple" vs "apples" and "bring" vs "brought", we can either stem or lemmatize the words. Stemming doesn't always result in actual words, for example "brought" might end up as "brough". Lemmatization produces whole actual words. We will see examples of lemmatization below, and later use lemmatization in our sklearn pipeline. You can read more about [SpaCy Tokens](https://spacy.io/api/token).

In [ ]:
# SpaCy models come with taggers, parsers and named-entity-recognizers.
# We will disable the parts of the spacy processing pipeline that we will not need.
# doc will be a series of SpaCy Tokens

doc = nlp(review_example, disable=["tagger", "parser", "ner"])

# You can think about a SpaCy token
# as a word in the doc with extra information
# coming from SpaCy processing
# like position in document, lemma, etc.

# For each token in the document
for token in doc:
    # if the token is not a stop word or a punctuation mark
    if not token.is_stop and not token.is_punct:
        # print the token text and its lemmatization
        print(f"{token.text:<30}{token.lemma_}")

<a id=sklearn-with-spacy></a>
## 4. Sklearn with SpaCy Lemmatization and Vocabulary

Finally, we can process the reviews with SpaCy lemmatization and vocabulary in the CountVectorizer. Then, run our MultinomialNB classifier on the output in a sklearn pipeline.

In [ ]:
import re


def preprocessor(text):
    """Cleans up html tags"""
    html_regex = "<[^>]*>*<[^>]*>"
    if type(text) == str:
        text = re.sub(html_regex, "", text)
        text = re.sub("[\W]+", "", text.lower())
    return text


# tokenize the doc, remove stop words and lemmatize its tokens
def tokenize(doc):
    tokens = nlp(doc)
    return [preprocessor(token.lemma_) for token in tokens if not token.is_stop]

The pipeline below will take significantly longer than the above pipeline where we had kept the default settings for the CountVectorizer. SpaCy can be optimized to run faster with nlp.pipe and processing all of the documents before inputting to CountVectorizer. We will not cover nlp.pipe here.

In [ ]:
spacy_pipeline = Pipeline(
    [
        (
            "vectorizer",
            CountVectorizer(
                vocabulary=nlp.vocab.strings, tokenizer=tokenize, ngram_range=(1, 1)
            ),
        ),
        ("classifier", MultinomialNB()),
    ]
)
spacy_pipeline.fit(X_train, y_train)

In [ ]:
predicted = spacy_pipeline.predict(X_test)
acc = accuracy_score(y_test, predicted)
print(acc)

CountVectorizer also has some processing steps such as converting to lowercase, which is set to True by default, and removing stop words. Check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) for other parameters.

<a id=multinomialnb-implementation></a>
## 5. Multinomial Naive Bayes Implementation

In [ ]:
from collections import defaultdict
from sklearn.base import BaseEstimator, ClassifierMixin


class NaiveBayes(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        """
        Args:
            X: a sparse matrix of counts
            y: Labels array
        Returns:
            self
        """

        # In the fit method we'll estimate the parameters of the model.
        # In sklearn, fit method in any class extended from BaseEstimator should
        # return the object itself.

        # In our pipeline, X will be the output of the CountVectorizer instance
        # and it is a sparse matrix.

        # We want to take out only the non-zero entries
        # and their coordinates (in document_number, word_index form)

        counts_positive_ = defaultdict(int)
        counts_negative_ = defaultdict(int)

        # counts_ will be a dictionary of dictionaries
        # It has two keys: 0 and 1
        # 0 will contain the counts of words in the negative documents
        # Similarly, 1 will contain the counts of words in the positive documents

        counts_ = {0: counts_negative_, 1: counts_positive_}
        class_counts_ = defaultdict(int)

        # dictionary that keeps the total number of words in all documents in each class
        total_word_counts = {0: 0, 1: 0}

        # estimate parameters of the model (P(w_i|c))
        # for each word w
        for d, w in zip(*X.nonzero()):
            # collect counts of w in every document
            w_count_in_d = X[d, w]

            # update the dictionary entries with the counts
            # from X.
            counts_[y[d]][w] += w_count_in_d

            # We also need the total word counts in both the positive and negative groups
            # Here, we are incrementing the total word count for the correct group.
            total_word_counts[y[d]] += w_count_in_d

        # the prior class probabilities
        positive_fraction = sum(y) / len(y)
        negative_fraction = 1 - positive_fraction

        # log class priors
        # max operation is needed to avoid invalid input to log
        self.class_priors_ = {
            0: np.log(negative_fraction),
            1: np.log(positive_fraction),
        }

        # We will calculate the class conditional probability (likelihood)
        # of each word.
        all_words = set(list(counts_[0].keys()) + list(counts_[1].keys()))

        # We initialize these likelihoods to 0
        self.word_probs_ = defaultdict(lambda: {0: 0, 1: 0})

        # then for each word, we'll update the word likelihoods
        # the likelihood for a word will be the count of that word in all documents in class c
        # divided by the count of all words in all documents in class c.
        for v in all_words:
            self.word_probs_[v][0] = np.log(max(1, counts_[0][v])) - np.log(
                total_word_counts[0]
            )
            self.word_probs_[v][1] = np.log(max(1, counts_[1][v])) - np.log(
                total_word_counts[1]
            )

        return self

    def predict(self, X, y=None):
        """
        Args:
            X: a sparse matrix of counts
            y: None (kept for compatibility)
        Returns:
            preds: an array of predicted classes
        """

        # For each document d in X,
        # we will use the class priors and likelihoods calculated in fit
        # to predict the predicted class for d.

        d = 0

        preds = []

        # unnormalized posterior probabilites
        prob_pos = self.class_priors_[1]
        prob_neg = self.class_priors_[0]

        prev_d = d

        # The sparse matrix structure of CountVectorizer output X
        # allows us to iterate over documents and words in X.

        for d, w in zip(*X.nonzero()):

            # When we encounter the next document in X,
            # append the class prediction for the previous document to preds
            # and reset the unnormalized posterior probability to the class prior.
            if d != prev_d:
                preds.append(1 if prob_pos > prob_neg else 0)
                prob_pos = self.class_priors_[1]
                prob_neg = self.class_priors_[0]
                prev_d = d

            # While we are still processing the same document,
            # add the word likelihoods to the unnormalized posterior probabilities.
            prob_pos += self.word_probs_[w][1]
            prob_neg += self.word_probs_[w][0]

        # append the class prediction for the last document
        preds.append(1 if prob_pos > prob_neg else 0)
        return preds

In [ ]:
pipeline = Pipeline(
    [
        (
            "vectorizer",
            CountVectorizer(
                vocabulary=nlp.vocab.strings, tokenizer=tokenize, ngram_range=(1, 1)
            ),
        ),
        ("classifier", NaiveBayes()),
    ]
)
pipeline.fit(X_train, y_train)

In [ ]:
preds = pipeline.predict(X_test)
accuracy_score(y_test, preds)

<a id=exercises></a>
## 6. Exercises

Another type of naive Bayes model known as **binary multinomial naive Bayes** (or binary naive Bayes) is commonly used for sentiment analysis. In this model, the likelihood $P(w_i \mid c)$ is given by the fraction of documents in class $c$ that contain $w_i$ rather than the fraction of all words in class $c$ that are the same as $w_i$.




#### 1. Adapt the classifier defined in [Multinomial naive Bayes implementation](#multinomialnb-implementation) to include add-one (Laplace) smoothing and plug into the pipeline instead of the MultinomialNB classifier. Report accuracy on the test set.  You can implement your own classifier from scratch instead of adapting the above.

**Hint:** Output of CountVectorizer instance has shape $n_{\text{doc}} \times n_{\text{vocab}}$.

#### 2. Remove duplicates from each review (`doc`). Remember `doc` variable is an object of type `spacy.Tokens.Doc`.  It is a sequence of `Token` objects. A token in SpaCy is more than a simple text and tokens for the same text are not equal to each other. You will need to compare either the `text` or better yet the `lemma_` attributes of the tokens.

#### After removing the duplicates, use the above pipeline (`spacy_pipeline`) to build a binary multinomial naive Bayes model. Report accuracy on the test set.


In [ ]:
# update the tokenize function to remove duplicates from each review (doc)
# it should return a list of lemmas as in the previous definition.
# A better way would be to define a preprocessor and pass it to the preprocessor
# parameter of CountVectorizer. To keep things simpler we will not be doing that
# here.

def tokenize(doc):
    # 
    pass
    
# Reconstruct the pipeline. 

spacy_pipeline = ...

# Fit the training set, make predictions for the test set and report accuracy.




<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

Please contact Zeynep Hakguder (<a href="mailto:zphakguder@gmail.com">zphakguder@gmail.com</a>) for further questions or inquries.